# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [1]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [2]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [3]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [4]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - a1885048


In [7]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [8]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [9]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [11]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_3", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
🔄 [todo_3] Research sleep improvement strategies (in_progress)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [12]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/rahul/Documents/AIE9/07_Deep_Agents/workspace


In [13]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
(empty directory)


In [14]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [15]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [16]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/rahul/Documents/AIE9/07_Deep_Agents/workspace


In [17]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## ✅ Your Personalized Sleep Improvement Plan is Complete!

I've created a comprehensive, evidence-based sleep improvement plan specifically tailored to your current challenges. The plan is saved as `/personalized_sleep_improvement_plan.md` and includes:

### 🎯 **Key Features of Your Plan:**

**Progressive 8-Week Structure:**
- **Weeks 1-2**: Establish consistent 11 PM bedtime (addressing your 10pm-1am variance)
- **Weeks 3-4**: Add morning light therapy (to combat morning fatigue)
- **Weeks 5-6**: Optimize sleep environment
- **Weeks 7-8**: Fine-tune and solidify habits

**Immediate Actions You Can Start Tonight:**
1. **Digital Sunset**: Phone goes to charging station outside bedroom at 10:30 PM
2. **Consistent Bedtime**: Target 11:00 PM (compromise within your current range)
3. **Replace Phone Time**: Use reading, journaling, or gentle stretching instead

### 🧠 **Why This Plan Will Work:**

**Addresses Your Specific Issues:**
- **Inconsistent bedtime** → Gradual shift

In [18]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
✅ [todo_1] Analyze current sleep issues (completed)
✅ [todo_3] Research evidence-based sleep hygiene practices (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Structure plan with timeline and actionable steps (completed)
✅ [todo_9] Save sleep plan to file (completed)
✅ [todo_6] Research sleep schedule consistency and circadian rhythm optimization (completed)
✅ [todo_8] Research technology/screen time management before bed (completed)
✅ [todo_10] Research sleep environment optimization (completed)
✅ [todo_12] Research pre-sleep routines and wind-down activities (completed)
✅ [todo_14] Research morning routines to improve energy levels (completed)
✅ [todo_16] Research gradual implementation strategies for habit change (completed)
✅ [todo_18] Compile comprehensive research summary (completed)


Workspace contents:
  [FILE] personalized_sleep_improvement_plan.md (6300 bytes)
  [DIR] research/
  [FILE] sleep_hygiene_research.md 

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
*Latency and Cost are two important consideration before using todo list for planning. Fact based questions (definitions/know how etc), questions with no clear goal, questions whihc don't require complex thinking deep agents are the scenarios where using todo list would be overkill. 

Granulairty of todo list should be grounded on if it can be completed, what would be updated/created as a result of same. Very high level/low granulaity todo (help improve sleep) and very low level/very high  granularity todo is probably a bad idea. It likely needs to be balances between the two limits with clear outcomes. 

That seems like a possibility depending on if LLM decides if task was completed w/o any gatekeeper ensuring all todo's are completed. There should be prompt to ensure LLM periodically checks that all todo's are completed and if for some reason any todo is not needed, a clear reasoning is provided.
*

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
*I will store the large health document in a filesystem and update prompt to read guide from fs rather than loading the file content in context window. 

Metrics -- I will store them in filesystem as well and a summary iof metrics is the only context which I will keep in context window.

Must remember -- this is what stays in context. Long term facts will also need to go in filesystem but should always be available in context window and must never be offloaded (things like medical condition, allergies etc)*

---
## 🏗️ Activity #1: Build a Research Agent

Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete

### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [33]:
### YOUR CODE HERE ###

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide

from langchain_core.tools import tool
from typing import List, Literal

# In-memory store for demo purposes
TODO_STORE = {}


@tool
def write_todos(todos: List[dict]) -> str:
    """
    Create a set of todos representing a multi-step research plan.

    Each todo should describe one meaningful unit of work
    (e.g., reading references, synthesizing findings, writing output).

    Args:
        todos: A list of todo objects.
               Each object must contain:
               - title (str): Short description of the task

    Returns:
        A confirmation message listing the created todo IDs.

    Example:
        write_todos([
            {"title": "Read wellness reference guide"},
            {"title": "Identify evidence-based strategies"},
            {"title": "Write structured markdown report"}
        ])
    """
    created_ids = []

    for todo in todos:
        todo_id = f"todo_{len(TODO_STORE) + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo["title"],
            "status": "pending"
        }
        created_ids.append(todo_id)

    return f"Created todos: {', '.join(created_ids)}"


@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """
    Update the execution status of an existing todo.

    This tool is used by the agent to explicitly track progress
    through the research workflow.

    Args:
        todo_id: The ID of the todo to update.
        status: The new status of the todo.
                Must be one of: pending, in_progress, completed.

    Returns:
        A confirmation message describing the update.

    Example:
        update_todo("todo_1", "completed")
    """
    if todo_id not in TODO_STORE:
        return f"Todo '{todo_id}' not found."

    TODO_STORE[todo_id]["status"] = status
    return f"{todo_id} updated to status '{status}'"


# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")


from pathlib import Path
import shutil

# Notebook-local reference to the guide
source_guide = Path("data/HealthWellnessGuide.txt").resolve()

# Agent workspace
workspace = Path("workspace").absolute()
(workspace / "data").mkdir(parents=True, exist_ok=True)
(workspace / "research_findings").mkdir(parents=True, exist_ok=True)

target_guide = workspace / "data" / "HealthWellnessGuide.txt"

# Copy guide into workspace exactly once
if not target_guide.exists():
    shutil.copy(source_guide, target_guide)

print(f"Guide available at: {target_guide}")


@tool
def write_research_file(filename: str, content: str) -> str:
    """
    Write a markdown research report inside research_findings/.

    This tool enforces:
    - relative paths only
    - fixed output directory
    - no directory creation
    - no absolute paths

    Args:
        filename: Markdown filename only (e.g. "stress_management_report.md")
        content: Markdown content to write

    Returns:
        Confirmation message with written file path
    """
    if "/" in filename or "\\" in filename:
        raise ValueError("Filename must not include path separators.")

    path = f"research_findings/{filename}"
    return write_file.invoke({
        "path": path,
        "content": content
    })

from deepagents.backends import FilesystemBackend

filesystem_backend = FilesystemBackend(
    root_dir=str(workspace),
    virtual_mode=True
)
# Step 3: Create the agent with a research-focused system prompt

from deepagents import create_deep_agent

research_agent = create_deep_agent(
    tools=[
        write_todos,
        update_todo,
        read_file,            
        write_research_file,  
    ],
    backend=filesystem_backend,
    system_prompt="""
You are a Deep Research Agent specializing in wellness topics.

You MUST follow this workflow:
1. Create todos outlining the research process.
2. Read data/HealthWellnessGuide.txt using read_file.
3. Identify at least 5 evidence-based stress management strategies.
4. Write a structured markdown report using write_research_file.
5. Update todo statuses as each step is completed.

Rules:
- Never choose file paths yourself.
- Never use absolute paths.
- Never inline large reference documents.
- The final output must be written to a markdown file.
- Todos are the source of truth for execution progress.
"""
)



# Step 4: Test with the stress management research task

result = research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": (
            "Research stress management techniques and create a comprehensive "
            "guide with at least 5 evidence-based strategies."
        )
    }]
})

print(result["messages"][-1].content)



Guide available at: /Users/rahul/Documents/AIE9/07_Deep_Agents/workspace/data/HealthWellnessGuide.txt
Perfect! I've successfully completed the research and created a comprehensive guide on stress management techniques. Here's what I've delivered:

## Summary

I've created a **comprehensive stress management guide** with **8 evidence-based strategies** (exceeding your requirement of 5), all drawn from the Health Wellness Guide reference document:

### The 8 Evidence-Based Strategies:

1. **Deep Breathing Techniques** - Box breathing method for immediate stress relief
2. **Progressive Muscle Relaxation (PMR)** - Systematic tension and release of muscle groups
3. **Mindfulness Meditation** - Multiple forms including focused attention, body scan, loving-kindness, and walking meditation
4. **Regular Physical Exercise** - Aerobic, strength training, and mind-body exercises
5. **Grounding Techniques** - 5-4-3-2-1 sensory method for acute stress
6. **Sleep Hygiene and Adequate Rest** - Essenti

In [37]:
print(ls.invoke({"path": "research_findings"}))
print(read_file.invoke({"path": "research_findings/stress_management_comprehensive_guide.md"}))


[FILE] stress_management_comprehensive_guide.md (22992 bytes)
[FILE] test.md (23 bytes)
# Comprehensive Guide to Evidence-Based Stress Management Techniques

## Executive Summary

Stress is the body's natural response to demands and threats, but chronic stress can significantly harm both physical and mental health. This comprehensive guide presents evidence-based stress management strategies drawn from established wellness research, offering both immediate relief techniques and long-term lifestyle interventions to help individuals effectively manage stress and improve overall well-being.

---

## Understanding Stress

### What is Stress?

Stress is the body's response to any demand or threat. While acute stress can be beneficial—motivating us to meet deadlines or respond to challenges—chronic stress can have serious negative impacts on health.

### Physical Symptoms of Stress
- Headaches and muscle tension
- Fatigue and low energy
- Sleep disturbances
- Digestive issues
- Rapid heartbe

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [38]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [39]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [40]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
Perfect! I've successfully created your comprehensive morning routine guide for better energy. Here's what I've accomplished:

## ✅ **Complete Project Summary**

### **1. Research Phase Completed**
- Conducted in-depth research on the science behind morning routines
- Covered circadian rhythm effects, exercise physiology, nutritional timing, and mindset practices
- Synthesized findings from key studies and research publications

### **2. Comprehensive Guide Created**
The guide includes everything you requested:

**🧪 Science Foundation:**
- Circadian rhythm connection and hormonal responses
- Exercise and metabolic activation benefits
- Nutritional timing and energy stability research
- Mindset-energy connection studies

**⚡ Practical 4-Phase Morning Routine (90 minutes):**
- **Phase 1**: Wake Up & Light Exposure (0-15 min)
- **Phase 2**: Movement & Exercise (15-45 min)  
- **Phase 3**: Nutrition & Hydration (45-75 min)
- **Phase 4**: Mindset & Mental Preparation (

In [42]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines and energy optimization (completed)
✅ [todo_2] Create a comprehensive morning routine guide with exercise, nutrition, and mindset components (completed)
✅ [todo_3] Format and save the guide as a well-structured markdown file (completed)

Generated files in workspace:
  [DIR] data/
  [FILE] morning_routine_guide.md (15312 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6300 bytes)
  [DIR] research/
  [DIR] research_findings/
  [FILE] science_based_morning_routine_guide.md (31413 bytes)
  [FILE] sleep_hygiene_research.md (17405 bytes)
  [FILE] sleep_improvement_quick_guide.md (4140 bytes)


## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [49]:
# Create memory-aware tools
from langgraph.store.base import BaseStore
from langgraph.store.memory import InMemoryStore

# Ensure memory_store exists (create if the store cell was not run first)
try:
    memory_store
except NameError:
    memory_store = InMemoryStore()

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [50]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [51]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Hi Alex! I don't have any previous information about your wellness goals, fitness level, or preferences yet. To recommend the best exercise routine for you, I'd like to learn more about:

1. **Current fitness level** - Are you a beginner, intermediate, or advanced exerciser?
2. **Goals** - Are you looking to lose weight, build muscle, improve cardiovascular health, increase flexibility, or something else?
3. **Time availability** - How many days per week and minutes per session can you commit to exercise?
4. **Equipment/location preferences** - Do you prefer home workouts, gym access, outdoor activities, or bodyweight exercises?
5. **Any physical limitations or health conditions** I should be aware of?
6. **Activities you enjoy or dislike** - What types of movement do you find fun or boring?

Once you share some of these details, I'll save them to your profile and create a personalized exercise routine that fits your lifestyle and goals. This way, I can provide even bet

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [52]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [53]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [54]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [55]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [56]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Complete! 

Based on your profile as a 35-year-old office worker looking to lose 15 pounds and improve sleep, I've created a comprehensive wellness assessment that identifies **sleep optimization** as your primary priority, since it directly impacts weight management, energy levels, and overall health.

### Key Findings:
- **Interconnected challenges**: Your sedentary work, sleep issues, and weight goals are all connected
- **Strong foundation**: No major health conditions and existing vegetarian diet choices
- **Biggest opportunity**: Improving sleep will create positive cascading effects

### Your Personalized Action Plan:

**🚀 Start TODAY:**
1. **Digital sunset** 1 hour before bed + 15-min wind-down routine
2. **Hourly movement breaks** during work (just 2 minutes each)
3. **Morning light exposure** + better hydration

**📅 Next 1-2 weeks:**
1. **Consistent sleep schedule** (even weekends)
2. **Desk exercise routine** 3x daily 
3. **Optimiz

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [58]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli version: deepagents 0.0.19


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [61]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [62]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [63]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! I've created your comprehensive 2-week wellness program, Alex! 🎉

## Here's what I've delivered for you:

### **4 Complete Reference Files:**

1. **🏃‍♂️ Exercise Plan** (`/user_alex_2_week_exercise_plan.txt`)
   - 3 workouts per week, 30 minutes each
   - Progressive structure from Week 1 to Week 2
   - Bodyweight focus with clear instructions
   - Tips for building consistency

2. **🥗 Vegetarian Nutrition Plan** (`/user_alex_2_week_nutrition_plan.txt`)
   - 14 days of complete meal planning
   - Vegetarian protein sources guide
   - Shopping lists and meal prep tips
   - Hydration and supplement recommendations

3. **🧘‍♀️ Stress & Sleep Program** (`/user_alex_2_week_mindfulness_plan.txt`)
   - Daily stress management techniques for work
   - Sleep optimization strategies
   - 10-20 minute daily practices
   - Emergency stress-busting tools

4. **📋 Comprehensive Integration Guide** (`/user_alex_comprehensive_2_week_wellness_program.txt`)
   - Day-by-da

In [64]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Save user preferences and create initial profile (completed)
✅ [todo_2] Create personalized exercise plan (3x/week, 30 mins) (completed)
✅ [todo_3] Develop vegetarian meal plan and nutrition guidance (completed)
✅ [todo_4] Design stress management and sleep optimization program (completed)
✅ [todo_5] Integrate all components into cohesive 2-week wellness program (completed)
✅ [todo_6] Save all plans as separate reference files (completed)

GENERATED FILES
  [DIR] data/
  [FILE] morning_routine_guide.md (15312 bytes)
  [FILE] personalized_sleep_improvement_plan.md (6300 bytes)
  [DIR] research/
  [DIR] research_findings/
  [FILE] science_based_morning_routine_guide.md (31413 bytes)
  [FILE] sleep_hygiene_research.md (17405 bytes)
  [FILE] sleep_improvement_quick_guide.md (4140 bytes)
  [FILE] user_alex_2_week_exercise_plan.txt (4818 bytes)
  [FILE] user_alex_2_week_mindfulness_plan.txt (9285 bytes)
  [FILE] user_alex_2_week_nutrition_plan.txt (6890 bytes)
  

In [65]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of personalized_sleep_improvement_plan.md:
# Personalized Sleep Improvement Plan

## Your Current Situation Analysis
- **Bedtime Range**: 10pm-1am (3-hour variance disrupting circadian rhythm)
- **Screen Habit**: Phone use in bed (blue light suppressing melatonin)
- **Morning State**: Tired despite sleep (indicating poor sleep quality)

## 8-Week Progressive Implementation Plan

### 🎯 PHASE 1: Foundation (Weeks 1-2)
**Primary Focus**: Establish consistent bedtime

#### Immediate Actions:
1. **Choose Target Bedtime**: 11:00 PM (compromise between your current range)
2. **Create Phone Station**: Designated charging spot outside bedroom
3. **Set Daily Alarms**: 
   - 10:30 PM: "Digital sunset - put phone away"
   - 10:45 PM: "Begin bedtime routine"
   - 11:00 PM: "Lights out"

#### Week 1-2 Routine:
- **10:30 PM**: Phone goes to charging station
- **10:30-11:00 PM**: Wind-down activities (reading, gentle stretching, journaling)
- **11:00 PM**: Lights out, no exceptions
- **Same 

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
*Subagents needs to be designed in mind considering separation of concerns. Each subagent ideally should be specialist solving specific need. For those specific needs tools should be distinct. Whereas there are certain collaboration that must happen so that their individual work response can be collated. So things like reading summaries -- tool used for that purpose should be shared tool.

While picking the model for each subagent, it's important to understand what that agent is suppose to solve. If solve is heavy research, we need to have better model (like claud sonet). Whereas if the need it say doing simple math or non complex agents -- we can leverage smaller, cheaper models like openai mini. 

I think granularity needs to be one cohesive unit of work - e.g Mental health agent, excercise specialist etc. It would not make sense to say break exercise specialist to different type of activites based agent -- thats the granuality we should avoid in my opinion.
*

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
*
Health advise must be grounded, explainable and conservative.
It's important to have persistent storage to keep log of necessary information about wellness goal, conditions and any other long term stable information.  
Ofcourse for user privacy, safety and compliance purposes we must utilize namespace to ensure we have user isolation.
For product like these and for that matter any solution, its important to keep trace log so monitoring and observability should be built in right from the inception of the Agent solution.
Cost management is another important aspect and must be monitored to ensure org remains within allocated budget and necessary finetunig/additional budget should be planned ahead of such runout of budget.
*

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [66]:
### YOUR CODE HERE ###

# Step 1: Define your subagent configurations

exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Designs safe, realistic exercise plans.",
    "system_prompt": """
You are an exercise specialist.

Output requirements:
- Create a 30-day exercise outline
- 3-4 sessions per week
- Each session ≤30 minutes
- Bullet format only
- No medical claims
""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Designs stress and sleep improvement routines.",
    "system_prompt": """
You are a mindfulness specialist.

Output requirements:
- 5-7 stress & sleep techniques
- Practical, low-effort
- Suitable for mild anxiety
- Bullet format only
""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}


# Step 2: Create any additional tools you need
from langchain_core.tools import tool
from datetime import date

@tool
def save_daily_checkin(user_id: str, notes: str) -> str:
    """
    Save a user's daily wellness check-in notes to a file.
    """
    today = date.today().isoformat()
    path = f"checkins/{user_id}_{today}.md"
    return write_file.invoke({
        "path": path,
        "content": f"# Daily Check-in ({today})\n\n{notes}"
    })


@tool
def write_weekly_summary(user_id: str, week: int, content: str) -> str:
    """
    Write a weekly wellness summary report.
    """
    filename = f"weekly_summary_week_{week}.md"
    return write_research_file.invoke({
        "filename": filename,
        "content": content
    })


# Step 3: Build the main coordinator agent
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    backend=filesystem_backend,
    subagents=[exercise_specialist, mindfulness_specialist],
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Memory
        get_user_profile,
        save_user_preference,
        # Context / persistence
        save_daily_checkin,
        write_weekly_summary,
        # Skills (optional, if you already implemented)
        load_skill,
    ],
    system_prompt="""
You are a 30-Day Wellness Coach.

You MUST:
1. Retrieve the user's profile at the start
2. Create todos for a 30-day wellness challenge
3. Delegate exercise planning to the exercise specialist
4. Delegate stress/sleep planning to the mindfulness specialist
5. Save plans and progress to files
6. Adapt recommendations based on user check-ins
7. Write a weekly summary every 7 days

Rules:
- Always respect stored preferences and conditions
- Use subagents for domain work
- Use files for daily notes and weekly summaries
- Update todos as progress is made
"""
)


# Step 4: Test with a user creating their 30-day challenge

TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """
My user_id is user_alex.

I want to start a 30-day wellness challenge.
Goals:
- Improve energy
- Reduce stress
- Stay consistent but not overwhelmed

I can exercise 3x per week for 30 minutes.
"""
    }]
})

print(result["messages"][-1].content)



# Step 5: Simulate a daily check-in and adaptation

checkin = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """
Day 3 check-in:

Exercise felt good.
Evening breathing helped.
Still feeling tired in the mornings.
"""
    }]
})

print(checkin["messages"][-1].content)



## 🎉 Your 30-Day Wellness Challenge is Ready!

**Welcome to Day 1, Alex!** I've created a comprehensive, personalized wellness plan designed specifically for your goals of improving energy, reducing stress, and staying consistent without feeling overwhelmed.

### What's Been Set Up:

**📋 Your Complete Challenge Package:**
- **Detailed Exercise Plan**: 4-week progression, 3x/week, 30-minute sessions
- **Daily Mindfulness Routines**: 5-15 minute practices with weekly themes
- **Comprehensive Challenge Guide**: Everything you need in one place
- **Daily Check-in System**: Track progress and patterns
- **Weekly Summary Reports**: Scheduled for Days 7, 14, 21, 28, and 30

### Today's Action Items:

**🧘‍♂️ Day 1 Mindfulness Practice:**
- 5-minute breathing exercise (inhale 4, hold 4, exhale 4, hold 4)
- Sit comfortably and focus on your breath

**💪 This Week's Exercise Focus:**
- Week 1 is about foundation and light activity
- Plan your 3 workout days (suggest: Mon/Wed/Fri or Tue/Thu/Sat)
- 

In [67]:
checkin = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """
Day 3 check-in:

Exercise felt good.
Evening breathing helped.
Still feeling tired in the mornings.
"""
    }]
})

print(checkin["messages"][-1].content)

## Day 3 Check-in Summary & Day 4 Action Plan

Great job on your Day 3 progress! Here's what I've noted:

### ✅ What's Working Well:
- **Exercise felt good** - You're building a positive association with movement
- **Evening breathing helped** - Your wind-down routine is effective

### 🎯 Focus Area: Morning Fatigue
I've created a comprehensive strategy to address your morning tiredness. The key insight is that morning energy is largely determined by sleep quality and circadian rhythm alignment.

### 🌅 Your Day 4 Action Plan:

**Tonight (Day 3 Evening):**
1. Start your wind-down at 8:00 PM (no screens)
2. Continue your evening breathing routine at 9:00 PM
3. Aim for lights-out by 10:00 PM
4. Place a full glass of water by your bedside

**Tomorrow Morning (Day 4):**
1. **Immediately upon waking:** Drink the entire glass of water
2. **5-minute gentle stretch:** Focus on neck, shoulders, spine
3. **2-minute energizing breathing** (4-7-8 method)
4. **Get natural light:** Step outside or by 

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)